<a href="https://colab.research.google.com/github/KamilShape/building_style_recognition/blob/main/computer_vision_my_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import random
import os
import numpy as np


### Connecting to google drive

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Getting Images

In [7]:
styles = ['greek-architecture', 'gothic-architecture', 'baroque-architecture', 'victorian-architecture', 'modern architecture']

In [8]:
def parse_url(content):
  soup = BeautifulSoup(content, 'html.parser')
  imgs = soup.find_all('img', {'class': 'grid-image'})
  return [img['src'] for img in imgs]

In [10]:
def get_page(style, page):
  for page in range(1, page + 1):
    url = f'https://www.freeimages.com/search/{style}/{page}'
    try:
      response = requests.get(url)
      content = response.content
      return content
    except:
      print('Error! Wrong url.')

In [11]:
def get_image(photo_url, path, file_name, style):
  image_response = requests.get(photo_url)
  image_content = image_response.content
  img = Image.open(BytesIO(image_content))
  # print(img.mode)
  if img.mode == 'RGB':
    os.makedirs(f'{path}/{style}', exist_ok=True)
    img.save(f'{path}/{style}/{file_name}', 'png')

In [12]:
train_path = 'drive/MyDrive/architecture_photos/data/train'

In [13]:
def get_photos(style, page, path):
  content = get_page(style, page)
  images = parse_url(content)
  for image in images:
    if len(image) > 5:
      file_name = image.split('/')[-1]
      get_image(image, train_path, file_name, style)


In [66]:
#DOWNLOADING ALL PHOTOS
for style in styles:
  get_photos(style, 10, train_path)

### Test and train data sets

In [67]:
test_path = 'drive/MyDrive/architecture_photos/data/test'

In [ ]:
os.makedirs(f'{test_path}')

In [68]:
def create_test_set(styles, train_path, test_path, set_size):
  for style in styles:
    files_folder = os.listdir(f'{train_path}/{style}')
    for n in range(0, round(len(files_folder)*set_size)+1):
      files_folder = os.listdir(f'{train_path}/{style}')
      photo_number = np.random.randint(len(files_folder))
      os.makedirs(f'{test_path}/{style}', exist_ok=True)
      os.replace(f'{train_path}/{style}/{files_folder[photo_number]}', f'{test_path}/{style}/{files_folder[photo_number]}')

In [69]:
create_test_set(styles, train_path, test_path, 0.15)

Model 

In [70]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow import keras

In [110]:
img_height = 256
img_width = 256
batch_size = 30

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    train_path,
    validation_split = 0.2,
    seed=0,
    subset='training',
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical')

val_ds = keras.utils.image_dataset_from_directory(
    train_path,
    validation_split = 0.2,
    seed=0,
    subset='validation',
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical')

In [115]:
model = Sequential()
model.add(layers.Rescaling(1./255))
model.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(16, (3,3), padding='same', activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

In [116]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

In [117]:
history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 10,
    batch_size = batch_size
)

Epoch 1/10
7/7 [==============================] - 6s 308ms/step - loss: 1.6747 - accuracy: 0.1837 - val_loss: 1.5921 - val_accuracy: 0.1667
Epoch 2/10
7/7 [==============================] - 1s 81ms/step - loss: 1.5909 - accuracy: 0.2347 - val_loss: 1.5789 - val_accuracy: 0.2500
Epoch 3/10
7/7 [==============================] - 1s 80ms/step - loss: 1.5032 - accuracy: 0.3112 - val_loss: 1.6114 - val_accuracy: 0.2500
Epoch 4/10
7/7 [==============================] - 1s 115ms/step - loss: 1.4074 - accuracy: 0.3878 - val_loss: 1.5282 - val_accuracy: 0.3750
Epoch 5/10
7/7 [==============================] - 1s 115ms/step - loss: 1.4194 - accuracy: 0.3980 - val_loss: 1.8048 - val_accuracy: 0.2500
Epoch 6/10
7/7 [==============================] - 1s 86ms/step - loss: 1.2926 - accuracy: 0.4337 - val_loss: 1.5769 - val_accuracy: 0.2500
Epoch 7/10
7/7 [==============================] - 1s 81ms/step - loss: 1.2199 - accuracy: 0.4898 - val_loss: 1.6004 - val_accuracy: 0.3750
Epoch 8/10
7/7 [========